In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio

In [ ]:
!pip install python_speech_features

In [4]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random
import operator
import math
from collections import defaultdict

In [5]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1
        else:
            classVote[response]=1
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

def distance(instance1 , instance2 , k ):
    distance =0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 ))
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance


In [13]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open(f'/content/drive/MyDrive/test/extract_feature.dat' , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

    for x in range(len(dataset)):
        if random.random() <split :
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])

trainSet = []
testSet = []
loadDataset(f'/content/drive/MyDrive/test/extract_feature.dat' , 0.70, trainSet, testSet)

leng = len(testSet)
predict = []
for x in range (leng):
    predict.append(nearestClass(getNeighbors(trainSet ,testSet[x] , 5)))

In [14]:
import gradio as gr

def process_audio(audio):

    #Prediction for new data
    results=defaultdict(int)
    i=1
    for folder in os.listdir( f'/content/drive/MyDrive/cluster'):
        results[i]=folder
        i+=1
    #print(results)
    (rate,sig)=wav.read(audio)
    mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
    covariance = np.cov(np.matrix.transpose(mfcc_feat))
    mean_matrix = mfcc_feat.mean(0)
    feature=(mean_matrix,covariance,0)

    pred=nearestClass(getNeighbors(dataset ,feature , 5))

    return results[pred]


In [15]:
inputs = gr.inputs.Audio(type='filepath')
output = gr.outputs.Textbox()


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:321: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:324: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(source=source, type=type, label=label, optional=optional)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


In [ ]:
gr.Interface(fn=process_audio, inputs=inputs, outputs=output).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://07c5f3dc7d67cbc887.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
